## 1. Import Packages

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Add, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, Layer, MaxPool2D, ReLU, Resizing
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import L2
from tqdm import tqdm

dataset_dir = 'C:\\Users\\Thomas Wang\\Desktop\\DSA5204 Project\\Dataset\\Facial Attractiveness\\SCUT-FBP5500_v2\\'
scut_dir = 'C:\\Users\\Thomas Wang\\Desktop\\DSA5204 Project\\Dataset\\Facial Attractiveness\\SCUT-FBP\\'
mebeauty_dir = 'C:\\Users\\Thomas Wang\\Desktop\\DSA5204 Project\\Dataset\\Facial Attractiveness\\MEBeauty\\'

In [2]:
print(tf.__version__)
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

2.10.0
Num GPUs Available 1


## 2. Dataset Cleaning

In [ ]:
# Remove Background from Images

mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation

file_list = glob.glob(dataset_dir + 'Images\\*')

BG_COLOR = (255, 255, 255)
with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
    for i in tqdm(range(len(file_list))):
        file_name = file_list[i].split('\\')[-1]
        image = cv2.imread(file_list[i])
        image_height, image_width, _ = image.shape

        results = selfie_segmentation.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        
        #fg_image = np.zeros(image.shape, dtype=np.uint8)
        #fg_image[:] = MASK_COLOR
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR

        output_image = np.where(condition, image, bg_image)
        cv2.imwrite(dataset_dir + 'Cleaned_Images\\' + file_name, output_image)

## 3. Dataset Preparation - MT-ResNet Full

In [ ]:
all_ratings = pd.read_csv(dataset_dir + 'train_test_files\\All_labels.txt', sep = ' ', header = None)
all_ratings.columns = ['img_path', 'rating']

img_arr = np.zeros([len(all_ratings), 224, 224, 3])
y_arr = np.zeros([len(all_ratings), 2])

for i in tqdm(range(len(all_ratings))):
    file_name = all_ratings.iloc[i, 0]
    
    race = file_name[0]
    gender = file_name[1]
    
    if gender == 'M':
        y_arr[i, 1] = 0
    elif gender == 'F':
        y_arr[i, 1] = 1
        
    y_arr[i, 0] = all_ratings.iloc[i, 1]
    
    img = tf.io.read_file(dataset_dir + 'Cleaned_Images\\' + file_name)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.keras.layers.Resizing(224, 224)(img)
    img /= 255
    img_arr[i] = img
    
np.save(dataset_dir + 'train_img_arr.npy', img_arr)
np.save(dataset_dir + 'train_y_arr.npy', y_arr)

In [3]:
train_img_arr = np.load(dataset_dir + 'train_img_arr.npy')
train_y_arr = np.load(dataset_dir + 'train_y_arr.npy')

## 4. Group Normalization Module

In [4]:
class GroupNormalization(Layer):
    def __init__(self, groups = 32, epsilon = 1e-5, **kwargs):
        super(GroupNormalization, self).__init__(**kwargs)
        self.groups = groups
        self.epsilon = epsilon
    
    
    
    def build(self, input_shape):
        self.gamma = self.add_weight(name = 'gamma', shape = (input_shape[-1],), initializer = 'ones', trainable = True)
        self.beta = self.add_weight(name = 'beta', shape = (input_shape[-1],), initializer = 'zeros', trainable = True)
    
    
    
    def call(self, inputs):
        input_shape = tf.shape(inputs)
        group_shape = [input_shape[i] for i in range(len(input_shape) - 1)]
        group_shape.append(input_shape[-1] // self.groups)
        group_shape.append(self.groups)
        group_shape = tf.stack(group_shape)
        
        x = tf.reshape(inputs, group_shape)
        mean, variance = tf.nn.moments(x, [1, 2], keepdims=True)
        x = (x - mean) / tf.sqrt(variance + self.epsilon)
        x = tf.reshape(x, input_shape)
        
        return self.gamma * x + self.beta
    
    
    
    def get_config(self):
        config = super(GroupNormalization, self).get_config()
        config.update({'groups': self.groups, 'epsilon': self.epsilon})
        return config

## 5. MT-ResNet Model

In [5]:
class Input_Stream(Model):
    def __init__(self, num_maps, kernel_size, pool):
        super(Input_Stream, self).__init__()
        
        self.conv1 = Conv2D(num_maps, kernel_size, strides = 2, use_bias = False)
        self.bn1 = GroupNormalization()
        self.conv2 = Conv2D(num_maps, kernel_size, padding = 'same', use_bias = False)
        self.bn2 = GroupNormalization()
        self.conv3 = Conv2D(num_maps, kernel_size, padding = 'same', use_bias = False)
        self.bn3 = GroupNormalization()
        self.relu = ReLU()
        self.max_pool = MaxPool2D(pool_size = pool, strides = 2)
        
       
    
    def call(self, inputs):
        x = self.relu(self.bn1(self.conv1(inputs)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.max_pool(x)
        
        return x

    

class ResBottleneckBlock(Model):
    def __init__(self, filters, downsample):
        super().__init__()
        
        self.downsample = downsample
        self.filters = filters
        self.conv1 = Conv2D(filters, 1, 1)
        if downsample:
            self.conv2 = Conv2D(filters, 3, 2, padding = 'same')
        else:
            self.conv2 = Conv2D(filters, 3, 1, padding = 'same')
        self.conv3 = Conv2D(filters * 4, 1, 1)
        self.gn1 = GroupNormalization()
        self.gn2 = GroupNormalization()
        self.gn3 = GroupNormalization()
        
        
    def build(self, input_shape):
        if self.downsample or self.filters * 4 != input_shape[-1]:
            self.shortcut = Sequential([
                Conv2D(
                    self.filters * 4, 1, 2 if self.downsample else 1, padding = 'same'),
                GroupNormalization()
            ])
        else:
            self.shortcut = Sequential()

            
            
    def call(self, input):
        shortcut = self.shortcut(input)

        input = self.conv1(input)
        input = self.gn1(input)
        input = ReLU()(input)

        input = self.conv2(input)
        input = self.gn2(input)
        input = ReLU()(input)

        input = self.conv3(input)
        input = self.gn3(input)
        input = ReLU()(input)

        input = input + shortcut
        return ReLU()(input)
    
    
    
class ResNet(Model):
    def __init__(self, repeat):
        super().__init__()
        '''
        self.layer0 = Sequential([
            Conv2D(64, 7, 2, padding='same'),
            MaxPool2D(pool_size = 3, strides = 2, padding = 'same'),
            GroupNormalization(),
            ReLU()
        ], name = 'layer0')
        '''
        
        self.input_conv = Input_Stream(num_maps = 64, kernel_size = 3, pool = 3)

        self.layer1 = Sequential([
            ResBottleneckBlock(64, downsample = False) for _ in range(repeat[0])
        ], name = 'layer1')

        self.layer2 = Sequential([
            ResBottleneckBlock(128, downsample = True)
        ] + [
            ResBottleneckBlock(128, downsample = False) for _ in range(1, repeat[1])
        ], name = 'layer2')

        self.layer3 = Sequential([
            ResBottleneckBlock(256, downsample = True)
        ] + [
            ResBottleneckBlock(256, downsample = False) for _ in range(1, repeat[2])
        ], name = 'layer3')

        self.layer4 = Sequential([
            ResBottleneckBlock(512, downsample = True)
        ] + [
            ResBottleneckBlock(512, downsample = False) for _ in range(1, repeat[3])
        ], name = 'layer4')

        self.gap = GlobalAveragePooling2D()
        self.fc1 = Dense(1, name = 'fap', use_bias = False)
        self.fc2 = Dense(1, activation = 'sigmoid', name = 'gender', use_bias = False)

        
        
    def call(self, x):
        x = self.input_conv(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.gap(x)
        x_fap = self.fc1(x)
        x_gender = self.fc2(x)
        
        return x_fap, x_gender
    
    

class MT_ResNet(ResNet):
    def __init__(self):
        super().__init__([3, 4, 6, 3])
        
        
        
    def call(self, input):
        return super().call(input)

## 6. Training Configuration & Setup

### 6.1 Loss Function Definition - MT-ResNet

In [6]:
Loss_Functions = {'output_1': 'mse', 'output_2': 'binary_crossentropy'}
Loss_Weights = {'output_1': 2, 'output_2': 1}

### 6.2 Callbacks Setup

In [7]:
def custom_scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.01) 

    
    
required_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True),
    tf.keras.callbacks.LearningRateScheduler(custom_scheduler),
    tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5, verbose = 1, cooldown = 2, min_lr = 0.0000001),
    tf.keras.callbacks.TerminateOnNaN()
]

### 6.3 Training Parameters Definition - MT-ResNet

In [8]:
adam_optimizer = Adam(learning_rate = 0.005)
model = MT_ResNet()
model.compile(optimizer = adam_optimizer, loss = Loss_Functions, loss_weights = Loss_Weights)

## 7. Training Phase - MT-ResNet

In [9]:
model.fit(train_img_arr, [train_y_arr[:, 0], train_y_arr[:, 1]], batch_size = 8, 
          epochs = 1000, verbose = 1, callbacks = required_callbacks, 
          validation_split = 0.2)

Epoch 1/1000
550/550 [==============================] - 115s 176ms/step - loss: 2.9734 - output_1_loss: 1.0229 - output_2_loss: 0.9277 - val_loss: 1.6601 - val_output_1_loss: 0.4149 - val_output_2_loss: 0.8303 - lr: 0.0050
Epoch 2/1000
550/550 [==============================] - 93s 169ms/step - loss: 1.9736 - output_1_loss: 0.6220 - output_2_loss: 0.7296 - val_loss: 3.0334 - val_output_1_loss: 1.2124 - val_output_2_loss: 0.6087 - lr: 0.0050
Epoch 3/1000
550/550 [==============================] - 94s 170ms/step - loss: 2.1577 - output_1_loss: 0.7076 - output_2_loss: 0.7425 - val_loss: 2.4908 - val_output_1_loss: 0.8666 - val_output_2_loss: 0.7576 - lr: 0.0050
Epoch 4/1000
550/550 [==============================] - 95s 173ms/step - loss: 1.7461 - output_1_loss: 0.5405 - output_2_loss: 0.6652 - val_loss: 1.3919 - val_output_1_loss: 0.4149 - val_output_2_loss: 0.5620 - lr: 0.0050
Epoch 5/1000
550/550 [==============================] - 94s 171ms/step - loss: 1.7399 - output_1_loss: 0.5508 -

Epoch 37/1000
550/550 [==============================] - 95s 172ms/step - loss: 0.4611 - output_1_loss: 0.1629 - output_2_loss: 0.1353 - val_loss: 0.6002 - val_output_1_loss: 0.1976 - val_output_2_loss: 0.2049 - lr: 0.0018
Epoch 38/1000
550/550 [==============================] - 95s 172ms/step - loss: 0.4234 - output_1_loss: 0.1530 - output_2_loss: 0.1175 - val_loss: 0.6303 - val_output_1_loss: 0.2117 - val_output_2_loss: 0.2068 - lr: 0.0018
Epoch 39/1000
550/550 [==============================] - 95s 172ms/step - loss: 0.3645 - output_1_loss: 0.1385 - output_2_loss: 0.0875 - val_loss: 0.6584 - val_output_1_loss: 0.2229 - val_output_2_loss: 0.2126 - lr: 0.0018
Epoch 40/1000
550/550 [==============================] - 95s 172ms/step - loss: 0.3368 - output_1_loss: 0.1319 - output_2_loss: 0.0731 - val_loss: 0.6521 - val_output_1_loss: 0.1964 - val_output_2_loss: 0.2593 - lr: 0.0018
Epoch 41/1000
550/550 [==============================] - 94s 172ms/step - loss: 0.3258 - output_1_loss: 0.12

In [10]:
model.save_weights('MT_ResNet_GN.h5')

## 8. SCUT-FBP Test Dataset Preparation and Evaluation

In [ ]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation

rating_df = pd.read_excel(scut_dir + 'Rating_Collection\\Attractiveness label.xlsx', header = 0)
img_arr = np.zeros([len(rating_df), 224, 224, 3])
y_arr = np.zeros([len(rating_df), 2])

for i in tqdm(range(len(rating_df))):
    num = rating_df.iloc[i, 0]
    img = cv2.imread(scut_dir + 'Data_Collection\\SCUT-FBP-' + str(num) + '.jpg')
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
        results = selfie_segmentation.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask, ) * 3, axis = -1) > 0.1
        bg_image = np.zeros(img.shape, dtype = np.uint8)
        bg_image[:] = (255, 255, 255)
        cleaned_img = np.where(condition, img, bg_image)
    
    cleaned_img = tf.keras.layers.Resizing(224, 224)(cleaned_img)
    cleaned_img /= 255
    img_arr[i] = cleaned_img
    
    y_arr[i, 0] = rating_df.iloc[i, 1]
    y_arr[i, 1] = 1
    
np.save(scut_dir + 'img_arr.npy', img_arr)
np.save(scut_dir + 'y_arr.npy', y_arr)

In [11]:
img_arr = np.load(scut_dir + 'img_arr.npy')
y_arr = np.load(scut_dir + 'y_arr.npy')

In [12]:
model.evaluate(img_arr, [y_arr[:, 0], y_arr[:, 1]])

16/16 [==============================] - 3s 139ms/step - loss: 1.0062 - output_1_loss: 0.2866 - output_2_loss: 0.4330


[1.0061659812927246, 0.28656190633773804, 0.4330422282218933]

## 9. MEBeauty Test Dataset Preparation

In [ ]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation

rating_df = pd.read_csv(mebeauty_dir + 'landmarks.csv', header = 0)
#img_arr = np.zeros([len(rating_df), 224, 224, 3])
#y_arr = np.zeros([len(rating_df), 2])

img_list = []
y_list = []

for i in tqdm(range(len(rating_df))):
    file_path = rating_df.iloc[i, 1]
    file_name = '\\'.join(file_path.split('/')[5:])
    gender = file_path.split('/')[5]
        
    try:
        img = cv2.imread(mebeauty_dir + 'cropped_images\\images_crop_align_mtcnn\\' + file_name)
        '''
        with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
            results = selfie_segmentation.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            condition = np.stack((results.segmentation_mask, ) * 3, axis = -1) > 0.1
            bg_image = np.zeros(img.shape, dtype = np.uint8)
            bg_image[:] = (255, 255, 255)
            cleaned_img = np.where(condition, img, bg_image)
        '''

        img = tf.keras.layers.Resizing(224, 224)(img)
        img /= 255
        #img_arr[i] = img
        img = img[np.newaxis, :, :, :]
        img_list.append(img)
        
        if gender == 'female':
            #y_arr[i, 1] = 1
            y_list.append(np.array([rating_df.iloc[i, 2] / 2, 1]).reshape(1, 2))
        else:
            #y_arr[i, 1] = 0
            y_list.append(np.array([rating_df.iloc[i, 2] / 2, 0]).reshape(1, 2))
            
        #y_arr[i, 0] = rating_df.iloc[i, 2]
    except:
        continue
    
img_arr = np.concatenate(img_list, axis = 0)
y_arr = np.concatenate(y_list, axis = 0)
    
np.save(mebeauty_dir + 'img_arr.npy', img_arr)
np.save(mebeauty_dir + 'y_arr.npy', y_arr)

In [13]:
img_arr = np.load(mebeauty_dir + 'img_arr.npy')
y_arr = np.load(mebeauty_dir + 'y_arr.npy')

In [18]:
new_model = MT_ResNet()
new_model.build((None, 224, 224, 3))
new_model.load_weights('MT_ResNet_GN.h5')
new_model.compile(optimizer = adam_optimizer, loss = Loss_Functions, loss_weights = Loss_Weights)

new_model.evaluate(img_arr, [y_arr[:, 0], y_arr[:, 1]])

73/73 [==============================] - 14s 138ms/step - loss: 4.2271 - output_1_loss: 1.2101 - output_2_loss: 1.8069


[4.227126598358154, 1.21011483669281, 1.806898832321167]